# Performance measurement in ExSeqProcessing (ver.1.6)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
import subprocess
import re
from IPython.display import display

pd.set_option('float_format', '{:f}'.format)

## top log

In [ ]:
proc_stat = pd.read_csv('summary-top-proc-status.csv',parse_dates=[0])
mem_stat = pd.read_csv('summary-top-mem-usage.csv',parse_dates=[0])

In [ ]:
proc_stat['elapsed_time'] = pd.to_timedelta(proc_stat['elapsed_time'])
proc_stat['proc'] = proc_stat['proc'].str.rstrip()

mem_stat['elapsed_time'] = pd.to_timedelta(mem_stat['elapsed_time'])

In [ ]:
mem_unit = mem_stat['mem_unit'].unique()[0]

### System memory status

In [ ]:
fig = mem_stat.plot(x='elapsed_time',y=['mem_used','buf_cache','avail_mem'],figsize=(10,5))
fig.set_ylabel('Memory usage (%s)' % mem_unit)

### MATLAB processes memory status

In [ ]:
fig = proc_stat.pivot_table(index='elapsed_time',values=['res','virt'],aggfunc=sum).plot(figsize=(10,5))
fig.set_ylabel('Memory usage (MiB)')

In [ ]:
proc_stat.pivot_table(index='elapsed_time',values=['res','virt'],aggfunc=sum).describe()

In [ ]:
fig = proc_stat.pivot_table(index='elapsed_time',columns=['proc'],values=['res'],aggfunc=sum).plot(figsize=(10,5))
fig.set_ylabel('Memory usage (MiB)')

### MATLAB processes cpu and memory usage (%)

In [ ]:
proc_usage = proc_stat.pivot_table(index='elapsed_time',values=['cpu'],aggfunc=sum)
fig = proc_usage.plot(alpha=0.5,figsize=(10,5))
proc_usage.rolling(window=12,center=True).mean().plot(ax=fig)
fig.set_title('CPU usage of all MATLAB processes')
fig.set_ylabel('CPU usage (%)')
fig.legend(['cpu','SMA cpu'])

proc_usage = proc_stat.pivot_table(index='elapsed_time',values=['mem'],aggfunc=sum)
fig = proc_usage.plot(alpha=0.5,figsize=(10,5))
proc_usage.rolling(window=12,center=True).mean().plot(ax=fig)
fig.set_title('Memory usage of all MATLAB processes')
fig.set_ylabel('Memory usage (%)')
fig.legend(['mem','SMA mem'])

In [ ]:
proc_usage_worker = proc_stat[proc_stat['proc'] == 'worker'].pivot_table(index='elapsed_time',values=['cpu'],aggfunc=sum)
fig = proc_usage_worker.plot(y=['cpu'],alpha=0.5,figsize=(10,5))
proc_usage_worker.rolling(window=12,center=True).mean().plot(ax=fig)
fig.set_title('CPU usage of MATLAB worker processes')
fig.set_ylabel('CPU usage (%)')
fig.legend(['cpu','SMA cpu'])

proc_usage_main = proc_stat[proc_stat['proc'] == 'main'].pivot_table(index='elapsed_time',values=['cpu'],aggfunc=sum)
fig = proc_usage_main.plot(y=['cpu'],alpha=0.5,figsize=(10,5))
proc_usage_main.rolling(window=12,center=True).mean().plot(ax=fig)
fig.set_title('CPU usage of MATLAB main process')
fig.set_ylabel('CPU usage (%)')
fig.legend(['cpu','SMA cpu'])

proc_usage_worker = proc_stat[proc_stat['proc'] == 'worker'].pivot_table(index='elapsed_time',values=['mem'],aggfunc=sum)
fig = proc_usage_worker.plot(y=['mem'],alpha=0.5,figsize=(10,5))
proc_usage_worker.rolling(window=12,center=True).mean().plot(ax=fig)
fig.set_title('Memory usage of MATLAB worker processes')
fig.set_ylabel('Memory usage (%)')
fig.legend(['mem','SMA mem'])

proc_usage_main = proc_stat[proc_stat['proc'] == 'main'].pivot_table(index='elapsed_time',values=['mem'],aggfunc=sum)
fig = proc_usage_main.plot(y=['mem'],alpha=0.5,figsize=(10,5))
proc_usage_main.rolling(window=12,center=True).mean().plot(ax=fig)
fig.set_title('Memory usage of MATLAB main process')
fig.set_ylabel('Memory usage (%)')
fig.legend(['mem','SMA mem'])

In [ ]:
proc_stat.pivot_table(index='elapsed_time',values=['cpu','mem'],aggfunc=sum).describe()

## nfsiostat log

In [ ]:
nfs_stat = pd.read_csv('summary-nfsiostat.csv',parse_dates=[0])

In [ ]:
nfs_stat['time'] = pd.to_timedelta(nfs_stat['time'])

### read/write throughput (kb/s)

In [ ]:
fig = nfs_stat[nfs_stat['fs-path']=='/mp/nas1'].pivot_table(index='time',values=['read-kb/s','write-kb/s']).plot(alpha=0.5,figsize=(10,5))
fig.set_ylabel('read/write (kb/s)')

In [ ]:
nfs_stat[nfs_stat['fs-path']=='/mp/nas1'].pivot_table(index='time',values=['read-kb/s','write-kb/s'],aggfunc=sum).describe()

### pages in readpages/writepages

In [ ]:
fig = nfs_stat[nfs_stat['fs-path']=='/mp/nas1'].pivot_table(index='time',values=['pages-in-readpages','pages-in-writepages']).plot(alpha=0.5,figsize=(10,5))
fig.set_ylabel('pages')

## vmstat log

In [ ]:
vm_stat = pd.read_csv('summary-vmstat.csv',parse_dates=[0])

In [ ]:
vm_stat['time'] = pd.to_timedelta(vm_stat['time'])

### block i/o

In [ ]:
vm_bio = vm_stat.pivot_table(index='time',values=['io-bi','io-bo'])
fig = vm_bio.plot(alpha=0.5,figsize=(10,5))
vm_bio.rolling(window=12,center=True).mean().plot(ax=fig)
fig.set_ylabel('blocks/s')
fig.legend(['io-bi','io-bo','SMA io-bi','SMA io-bo'])

In [ ]:
vm_stat.pivot_table(index='time',values=['io-bi','io-bo']).describe()

### cpu status

In [ ]:
fig = vm_stat.pivot_table(index='time',values=['cpu-us','cpu-sy','cpu-id','cpu-wa','cpu-st']).plot(alpha=0.5,figsize=(10,5))
fig.set_ylabel('CPU usage (%)')

In [ ]:
vm_stat.pivot_table(index='time',values=['cpu-us','cpu-sy','cpu-id','cpu-wa','cpu-st']).describe()

## iostat log

In [ ]:
io_stat = pd.read_csv('summary-iostat.csv',parse_dates=[0])

In [ ]:
io_stat['time'] = pd.to_timedelta(io_stat['time'])

In [ ]:
comp_disk = re.compile('^([a-z][^ ]*) .* disk')
comp_dir = re.compile('(/mp/.*)$')

dev_dir_map = {}
for line in open('lsblk.txt'):
    line = line.rstrip()
    if line.startswith('NAME'):
        continue
#    print(line)
    
    m = comp_disk.search(line)
    if m != None:
        dev_name = m.group(1)
#        print(dev_name)
        continue
        
    m = comp_dir.search(line)
    if m != None:
        dir_name = m.group(1)
        dev_dir_map[dev_name] = dir_name

In [ ]:
dev_list = io_stat[~io_stat['device'].str.contains('dm-')]['device'].unique()
mp_list = [dev_dir_map[str] if str in dev_dir_map else str for str in dev_list]
dev_mp_map = dict(zip(dev_list,mp_list))
print(dev_mp_map)

In [ ]:
io_stat['device'].replace(dev_mp_map,inplace=True)

### device read/write (kB/s)

In [ ]:
for d in np.unique(mp_list):
    dev_rw = io_stat[io_stat['device'] == d].pivot_table(index=['time'],values=['rkB/s','wkB/s'],aggfunc=sum)
    dev_rw_sum = dev_rw[:].sum()
    if dev_rw_sum['rkB/s'] == 0 and dev_rw_sum['wkB/s'] == 0:
        continue

    fig = dev_rw.plot(alpha=0.5,figsize=(10,5))
    dev_rw.rolling(window=12,center=True).mean().plot(ax=fig)
    fig.set_title('device='+d)
    fig.set_ylabel('kB/s')
    fig.legend(['rkB/s','wkB/s','SMA rkB/s','SMA wkB/s'])

In [ ]:
dev_rw = io_stat[io_stat['device'].isin(mp_list)].pivot_table(index=['time'],columns=['device'],values=['rkB/s','wkB/s'],aggfunc=sum)
dev_rw.describe()

### device util (%)

In [ ]:
for d in np.unique(mp_list):
    dev_util = io_stat[io_stat['device'] == d].pivot_table(index=['time'],values=['%util'],aggfunc=sum)
    dev_util_sum = dev_util[:].sum()
    if dev_util_sum['%util'] == 0:
        continue

    fig = dev_util.plot(alpha=0.5,figsize=(10,5))
    dev_util.rolling(window=12,center=True).mean().plot(ax=fig)
    fig.set_title('device='+d)
    fig.set_ylabel('util (%)')
    fig.legend(['%util','SMA %util'])

In [ ]:
dev_util = io_stat[io_stat['device'].isin(mp_list)].pivot_table(index=['time'],columns=['device'],values=['%util'],aggfunc=sum)
dev_util.describe()

## gpu log

In [ ]:
gpu_stat = None
if os.path.exists('summary-gpu.csv'):
    gpu_stat = pd.read_csv('summary-gpu.csv',parse_dates=[0])

In [ ]:
gpu_stat['time'] = pd.to_timedelta(gpu_stat['time'])

In [ ]:
if gpu_stat is not None:
    gpu_util = gpu_stat.pivot_table(index='time',values=['utilization.gpu [%]','utilization.memory [%]'],aggfunc=sum)
    fig = gpu_util.plot(alpha=0.5,figsize=(10,5))
    gpu_util.rolling(window=10,center=True).mean().plot(ax=fig)
    fig.set_ylabel('util (%)')
    fig.legend(['gpu','gpu memory','SMA gpu','SMA gpu memory'])
    
    gpu_util_each = gpu_stat.pivot_table(index=['time'],columns=['index'],values=['utilization.gpu [%]','utilization.memory [%]']).fillna(0.0)
    fig = gpu_util_each.plot(alpha=0.5,figsize=(10,5))
    fig.set_ylabel('util (%)')

In [ ]:
if gpu_stat is not None:
    gpu_util = gpu_stat.pivot_table(index='time',values=['utilization.gpu [%]','utilization.memory [%]'],aggfunc=sum)
    display(gpu_util.describe())

## Summary

In [ ]:
ds1 = proc_stat.pivot_table(index='elapsed_time',values=['cpu','mem'],aggfunc=sum).describe()

if gpu_stat is not None:
    ds2 = gpu_stat.pivot_table(index='time',values=['utilization.gpu [%]'],aggfunc=sum).describe()
    ds = pd.concat([ds1, ds2],axis=1)

print(ds.loc[['mean','max'],['mem','cpu','utilization.gpu [%]']])
print(' ')
    
dev_ds = io_stat[io_stat['device'].isin(mp_list)].pivot_table(index=['time'],columns=['device'],values=['rkB/s','wkB/s','%util'],aggfunc=sum).describe()
print(dev_ds.loc[['mean','max'],['rkB/s','wkB/s']])
print(dev_ds.loc[['mean','max'],['%util']])